In [ ]:
!pip install openai


In [16]:

# BLOCK 0 — INITIALIZATION

import os
import sys
import requests
import sqlite3
import importlib.metadata
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# 1 Set project root
project_root = os.path.abspath("..")
sys.path.append(project_root)

print("Project root:", project_root)

# 2 Load environment variables
load_dotenv(os.path.join(project_root, ".env"))

# 3 Spotify Authentication (cache in project root)
scope = (
    "playlist-read-private "
    "playlist-modify-private "
    "playlist-modify-public "
    "user-library-read"
)

cache_path = os.path.join(project_root, ".spotify_cache")

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope=scope,
        cache_path=cache_path
    )
)

print("Spotify authenticated.")
print("Current user:", sp.current_user()["id"])

# 4 Database connection
from core.database import get_connection, create_tables

db_path = os.path.join(project_root, "music_agent.db")
conn = get_connection(db_path)

create_tables(conn)

print("Database connected.")
print("SQLite version:", sqlite3.sqlite_version)

# 5 Verify tables exist
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

print("Spotipy version:", importlib.metadata.version("spotipy"))

# Initial Sync

from core.ingestion import sync_new_tracks
from core.database import get_latest_added_at

new_tracks = sync_new_tracks(sp, conn, get_latest_added_at)
print("New tracks inserted:", new_tracks)

# 7 Verify counts
cursor.execute("SELECT COUNT(*) FROM tracks;")
print("Total tracks:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM albums;")
print("Total albums:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM track_albums;")
print("Total track_album relations:", cursor.fetchone()[0])

Project root: /Users/nazb/VSCode101/Personal-music-architec
Spotify authenticated.
Current user: 21ogollmd4ljapwluvwxe3stq
Database connected.
SQLite version: 3.45.3
Tables: [('tracks',), ('artists',), ('albums',), ('track_artists',), ('track_albums',)]
Spotipy version: 2.25.2
New tracks inserted: 0
Total tracks: 721
Total albums: 597
Total track_album relations: 721


In [ ]:

# BLOCK 1 — INITIAL FULL SYNC


from core.ingestion import sync_new_tracks
from core.database import get_latest_added_at

new_tracks = sync_new_tracks(sp, conn, get_latest_added_at)

print("New tracks inserted:", new_tracks)

# Verify total count
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM tracks;")
total_tracks = cursor.fetchone()[0]

print("Total tracks in database:", total_tracks)


In [ ]:

# BLOCK 2 — CREATE PLAYLIST


from core.playlists import get_tracks, create_playlist, add_tracks_to_playlist

# 1 Get Eminem tracks from DB
eminem_tracks = get_tracks(conn, artist="Eminem")

print("Eminem tracks found:", len(eminem_tracks))

track_ids = [track[0] for track in eminem_tracks]

# 2 Create playlist
playlist_name = f"Eminem - {len(track_ids)}"

playlist_id = create_playlist(sp, playlist_name)

print("Playlist created:", playlist_name)
print("Playlist ID:", playlist_id)

# 3 Add tracks
add_tracks_to_playlist(sp, playlist_id, track_ids)

print("Tracks added to playlist.")


In [ ]:
from core.playlists import remove_tracks_from_playlist

# Seleccionamos otra canción distinta
track_to_remove = track_ids[1]

result = remove_tracks_from_playlist(
    sp,
    playlist_id,
    [track_to_remove]
)

print("Track removed:", track_to_remove)
print("API response:", result)


In [ ]:
from core.playlists import update_playlist_details

new_name = "Eminem - Updated"

update_playlist_details(
    sp,
    playlist_id,
    name=new_name
)

print("Playlist renamed to:", new_name)


In [ ]:
from core.playlists import unfollow_playlist

unfollow_playlist(sp, playlist_id)

print("Playlist unfollowed (deleted).")


In [ ]:
playlist_data = sp.playlist(playlist_id)

print("Playlist name:", playlist_data["name"])
print("Total tracks:", playlist_data["items"]["total"])


In [ ]:
from core.playlists import get_playlist_items

items = get_playlist_items(sp, playlist_id)

print("Items returned:", len(items["items"]))

for item in items["items"][:3]:
    print(item["item"]["name"])


In [ ]:
from core.playlists import get_current_user_playlists

playlists = get_current_user_playlists(sp)

print("Playlists returned:", len(playlists["items"]))

for p in playlists["items"][:5]:
    print(p["name"])


In [ ]:
import requests

def get_playlist_items(sp, playlist_id, limit=100, offset=0):
    """
    Retrieve playlist items using the official /items endpoint.
    Avoids deprecated /tracks endpoint used internally by older Spotipy versions.
    """
    token = sp.auth_manager.get_access_token(as_dict=False)

    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/items"

    headers = {
        "Authorization": f"Bearer {token}"
    }

    params = {
        "limit": limit,
        "offset": offset
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        raise RuntimeError(
            f"Failed to retrieve playlist items. Status: {response.status_code}, Response: {response.text}"
        )

    return response.json()


In [ ]:
items = get_playlist_items(sp, playlist_id)

print("Items returned:", len(items["items"]))

for item in items["items"][:3]:
    print(item["item"]["name"])


In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-mini",
    input="Di hola en una frase corta."
)

print(response.output_text)


¡Hola, qué tal!


In [ ]:
tools = [
    {
        "type": "function",
        "name": "create_playlist",
        "description": "Create a new Spotify playlist for the current user.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the playlist to create."
                }
            },
            "required": ["name"]
        }
    }
]


In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Crea una playlist llamada Chill Nocturno",
    tools=tools
)

print(response.output)


In [ ]:
tools = [
    {
        "type": "function",
        "name": "create_playlist",
        "description": "Create a new Spotify playlist when the user explicitly asks to create one.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the new playlist."
                }
            },
            "required": ["name"],
            "additionalProperties": False
        }
    }
]


In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Crea una playlist llamada Chill Nocturno",
    tools=tools
)

print(response.output)


In [ ]:
for item in response.output:
    print("TYPE:", item.type)
    if item.type == "function_call":
        print("NAME:", item.name)
        print("ARGS:", item.arguments)
    else:
        print("TEXT:", getattr(item, "content", None))


In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Haz algo con música",
    tools=tools
)

print(response.output)


In [ ]:
create_playlist_tool = {
    "type": "function",
    "name": "create_playlist",
    "description": "Create a new Spotify playlist when the user explicitly asks to create one.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Name of the new playlist."
            }
        },
        "required": ["name"],
        "additionalProperties": False
    }
}


In [ ]:
rename_playlist_tool = {
    "type": "function",
    "name": "rename_playlist",
    "description": "Rename an existing Spotify playlist when the user wants to change its name.",
    "parameters": {
        "type": "object",
        "properties": {
            "playlist_name": {
                "type": "string",
                "description": "Current name of the playlist."
            },
            "new_name": {
                "type": "string",
                "description": "New name for the playlist."
            }
        },
        "required": ["playlist_name", "new_name"],
        "additionalProperties": False
    }
}


In [ ]:
tools = [create_playlist_tool, rename_playlist_tool]


In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Crea una playlist llamada Road Trip",
    tools=tools
)

print(response.output)


In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Renombra la playlist Chill a Chill 2026",
    tools=tools
)

print(response.output)


In [ ]:
LEVEL_1_TOOLS = [
    {
        "type": "function",
        "name": "create_artist_playlist",
        "description": "Create a new playlist containing all songs from a specific artist in the user's local library.",
        "parameters": {
            "type": "object",
            "properties": {
                "artist_name": {"type": "string"},
                "playlist_name": {"type": "string"}
            },
            "required": ["artist_name"]
        }
    }
]


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Create a playlist with my Drake songs",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


In [ ]:
import json

tool_call = response.output[0]
arguments = json.loads(tool_call.arguments)

result = create_artist_playlist(conn, sp, **arguments)
print(result)


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add my recent 3 songs to Travel",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Create a playlist with my Drake songs",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


In [6]:
from core.llm_tools import LEVEL_1_TOOLS

In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Create a playlist with my Drake songs",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add my recent 3 songs to Travel",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Remove all Rihanna songs from Gym",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Rename Travel to Summer 2025",
    tools=LEVEL_1_TOOLS,
)

print(response.output)


fase 2

In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add 2 Eminem songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add the album Scorpion to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


In [10]:
def validate_tool_call(tool_name, arguments):
    """
    Level 1 Conservative Validation
    Returns:
        {"valid": True}
        or
        {"valid": False, "message": "..."}
    """
    
    return {"valid": True}


In [5]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add my last 5 songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


[ResponseFunctionToolCall(arguments='{"playlist_name":"Travel","criteria":{"recent_limit":5}}', call_id='call_t4pNJCu9tgxPSxryGNdR1oni', name='add_tracks_to_playlist_by_name', type='function_call', id='fc_0b5dc9872708627c006995f121aa2c8196ace9f80492198cc7', status='completed')]


In [6]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add 3 Drake songs and the album Scorpion to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


[ResponseFunctionToolCall(arguments='{"playlist_name":"Travel","criteria":{"artists":["Drake"],"albums":["Scorpion"]}}', call_id='call_V41rR0zuMZiNkyT5O5Qlz6EI', name='add_tracks_to_playlist_by_name', type='function_call', id='fc_0365232a4fa97556006995fd46f8d4819486801df6d2b0f455', status='completed')]


In [7]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add 5 songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


[ResponseOutputMessage(id='msg_08295a6abffa6b04006995fdc6a6f8819786d108635b340bb4', content=[ResponseOutputText(annotations=[], text='Could you clarify what kind of songs you\'d like me to add to the "Travel" playlist? For example, do you want recent songs, specific artists, albums, or any other criteria? Please specify, so I can add the right tracks for you.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]


In [8]:
response = client.responses.create(
    model="gpt-4.1",
    input="Create a playlist with 10 songs",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


[ResponseOutputMessage(id='msg_00d5b4380867c681006995fdd75b8c819488011d4e0f3fdb04', content=[ResponseOutputText(annotations=[], text='Could you please provide more details on how you’d like me to select the 10 songs? For example, should they be from a specific artist, album, genre, or are you interested in your 10 most recently added songs, random songs, or any other criteria? Let me know your preference so I can create the playlist just the way you want!', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]


In [9]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add some Drake songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print(response.output)


[ResponseFunctionToolCall(arguments='{"playlist_name":"Travel","criteria":{"artists":["Drake"]}}', call_id='call_M8EYEjyno0UqZOYUZOre2lWt', name='add_tracks_to_playlist_by_name', type='function_call', id='fc_0d58c98b10b481be006995fdee46c88190bd39ce7b28be8da1', status='completed')]


In [7]:
import json
from openai import OpenAI

client = OpenAI()

def validate_tool_call(tool_name, arguments):
    """
    Level 1 Conservative Validation
    """

    # Solo validamos tools con criteria
    if tool_name in [
        "add_tracks_to_playlist_by_name",
        "remove_tracks_from_playlist_by_name",
        "create_mixed_playlist"
    ]:

        criteria = arguments.get("criteria", {})

        # 🔒 Regla 1:
        # Si hay artists pero no hay artist_limits → inválido
        if "artists" in criteria and "artist_limits" not in criteria:
            return {
                "valid": False,
                "message": "You must specify how many songs per artist."
            }

    return {"valid": True}

# ---------------------------------------
# Test Input
# ---------------------------------------
response = client.responses.create(
    model="gpt-4.1",
    input=" Add 3 Drake songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print("Model Output:")
print(response.output)


# ---------------------------------------
# Validation Layer (inspection only)
# ---------------------------------------
output = response.output[0]

if output.type == "function_call":
    tool_name = output.name
    arguments = json.loads(output.arguments)

    validation = validate_tool_call(tool_name, arguments)

    print("\nValidation Result:")
    print(validation)

else:
    print("\nNo tool call detected.")


Model Output:
[ResponseFunctionToolCall(arguments='{"playlist_name":"Travel","criteria":{"artists":["Drake"]}}', call_id='call_9fsA3ON0nQT07H6rntpC7XJ2', name='add_tracks_to_playlist_by_name', type='function_call', id='fc_00381a239f7708e900699602f96d248193804a8f8530aaaf71', status='completed')]

Validation Result:
{'valid': False, 'message': 'You must specify how many songs per artist.'}


In [10]:
import json
from openai import OpenAI

client = OpenAI()

def validate_tool_call(tool_name, arguments):
    if tool_name in [
        "add_tracks_to_playlist_by_name",
        "remove_tracks_from_playlist_by_name",
        "create_mixed_playlist"
    ]:
        criteria = arguments.get("criteria", {})

        if not criteria:
            return {
                "valid": False,
                "message": "Selection criteria is required."
            }

        if "artists" in criteria and "artist_limits" not in criteria:
            return {
                "valid": False,
                "message": "You must specify how many songs per artist."
            }

    return {"valid": True}

response = client.responses.create(
    model="gpt-4.1",
    input="Add 3 Drake songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print("Model Output:")
print(response.output)

output = response.output[0]

if output.type == "function_call":
    tool_name = output.name
    arguments = json.loads(output.arguments)
    validation = validate_tool_call(tool_name, arguments)
    print("Validation Result:")
    print(validation)
else:
    print("No tool call detected.")


Model Output:
[ResponseFunctionToolCall(arguments='{"playlist_name":"Travel","criteria":{"artists":["Drake"]}}', call_id='call_O65u6dnFkPezNlzwO1H9dfcf', name='add_tracks_to_playlist_by_name', type='function_call', id='fc_007146f54fb1be06006996ca59c2408193bbc4d3d2e583606c', status='completed')]
Validation Result:
{'valid': False, 'message': 'You must specify how many songs per artist.'}


In [11]:
import json
from openai import OpenAI

client = OpenAI()

def validate_tool_call(tool_name, arguments):
    if tool_name in [
        "add_tracks_to_playlist_by_name",
        "remove_tracks_from_playlist_by_name",
        "create_mixed_playlist"
    ]:
        criteria = arguments.get("criteria", {})

        if not criteria:
            return {
                "valid": False,
                "message": "Selection criteria is required."
            }

        if "artists" in criteria and "artist_limits" not in criteria:
            return {
                "valid": False,
                "message": "You must specify how many songs per artist."
            }

    return {"valid": True}

response = client.responses.create(
    model="gpt-4.1",
    input="Add 3 Drake songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

output = response.output[0]

if output.type == "function_call":
    tool_name = output.name
    arguments = json.loads(output.arguments)

    validation = validate_tool_call(tool_name, arguments)

    if validation["valid"]:
        print("Ready to execute:", tool_name, arguments)
    else:
        print(validation["message"])
else:
    print(output.content[0].text)


You must specify how many songs per artist.


In [12]:
import json
from core.execution_policy import ExecutionPolicy
from openai import OpenAI

client = OpenAI()


In [21]:
response = client.responses.create(
    model="gpt-4.1",
    input="Add 5 songs to Travel",
    tools=LEVEL_1_TOOLS,
    parallel_tool_calls=False
)

print("Model Output:")
print(response.output)

output = response.output[0]

if output.type == "function_call":
    tool_name = output.name
    arguments = json.loads(output.arguments)

    validation = ExecutionPolicy.validate(tool_name, arguments)

    print("Validation Result:")
    print(validation)
else:
    print("No tool call detected.")


Model Output:
[ResponseOutputMessage(id='msg_05e0533ec07ae3460069972cc0aa908195ad99a150751f87ee', content=[ResponseOutputText(annotations=[], text='To proceed, could you clarify what kind of songs you\'d like to add to the "Travel" playlist? For example, do you prefer:\n\n- Recently added songs\n- Songs from a specific artist or album\n- Specific song titles\n- Any five random songs\n\nPlease specify your preference so I can add the right tracks!', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
No tool call detected.
